# Flask API

We are going to build the Flask Python App that will make use of our neural network Python class.

Let's first create a file called app.py.

To get started, let's import some of the libraries we are going to use.

In [ ]:
from flask import Flask, redirect, render_template, request, session, url_for, send_from_directory, jsonify
from nn_model import NN_model
from rr_model import RR_model
from werkzeug import secure_filename
from configparser import SafeConfigParser
from logger import Logger
import pickle as pkl
import os

Most of these libraries come with the Anaconda and/or Python installation package, but there are some libraries such as Flask and werkzeug you will have to download yourself.

### Logger Setup

Let's setup our logger, in this case the logger is directing all the log lines to main_app.log. It is always good practice to name your log files with heart otherwise it will be very hard to debug.

In [ ]:
log = Logger('main_app.log')
log.info('Starting Application')

### Reading from a config file

Config files are to help you to give meaning to the "magic" numbers you defined for your model or class. You can think of it as a global variable that is read from a file with the extension of .ini

In [ ]:
config = SafeConfigParser()
config.read('config.ini')
UPLOAD_FOLDER = config.get('UploadFolders','UPLOAD_FOLDER_PRED')
UPLOAD_FOLDER_TRAIN = config.get('UploadFolders','UPLOAD_FOLDER_TRAIN')
DOWNLOAD_FOLDER = config.get('DownloadFolders','DOWNLOAD_FOLDER')
ALLOWED_EXT = set(['csv'])

### Setting Flask Variables

Flask variables are needed to run the app successfully. In the code below, the Flask app is declared and the upload folder for any file is specified.
You should notice there is a secret key. The secret key is needed for implementing accounts for login, so you can remove it if you are not planning to implement login functions.

In [ ]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.secret_key = os.urandom(12)

### Setting up Model variables

The variables we are going to set are global variables used to store the active model, temporary model, and the secondary model.

In [ ]:
model = None
temp_model = None
secondary_model = None

### Pickling Setup

In the below lines, a pickle file name is decided then we check if there is an existing pickle file with that name. If there is one, we initialize our active model with that pickle file, if not we create a brand new model class.

In [ ]:
fname = 'nn_model.pkl'
if os.path.isfile(fname):
	file_opened = open(fname, "rb")
	model = pkl.load(file_opened)
	file_opened.close()
else:
	model = NN_model()

# Now this is where the meat and patatoes of the app is. We are going to implement the functions that our HTML will call to interact with our model

## Flask Route Setup Part 1

In this part we are going to setup some functions to help us do the initial redirect and to download the files.

In [ ]:
@app.route('/')
def home():
	session['changedmodel' = 'False'
	return redirect(url_for('index_train'))

@app.route('/download', methods=['GET'])
def download(filename_1):
	try:
		abs_path = app.root_path + DOWNLOAD_FOLDER
		return send_from_directory(abs_path,filename=filename_1,as_attachment=True)
	except:
		errorString = 'Unable to download file'
		return render_template('predict_submission.html',errorOcurred='True',errorInfo=errorString)
	return render_tempalte('predict_submission.html')
	



The home function is for the initial redirect as we made custom URL for each function. THe download function takes a file name as parameter and downloads that file from the selected path as an attachment

## Flask Route Setup Part 2

In this part we are going to set up the changing model mechanism. It simply takes the active model, change that to the secondary_model, then change the secondary_model to the active model.

In [ ]:
@app.route('/change_model',methods=['GET','POST'])
def change_model():
	global model, temp_model,secondary_model
	session['changedmodel'] = 'True'
	if secondary_model is not None:
		temp = secondary_model
		secondary_model = model
		model = temp
	else:
		secondary_model = model
		model = RR_model()
	return redirect(url_for('index_train'))



## Flask Route Setup Part 3

In this part, we are going to implement the predictions function. The predict function will check if the file meets the requirements to be sent to the model class. Once everything checks out it will save the uploaded file and then pass it to the model class method, where it is then downloaded.

In [ ]:
@app.route('/predict',methods=['GET'])
def predict():
	if request.method == 'POST' and request.files['file']:
		file_to_process = request.files['file']
		filename = secure_filename(file_to_process.filename)
		file_to_process.save(os.path.join(app.config['UPLOAD_FOLDER'],filename))
		return download(model.predict(file_to_process.filename))
	return render_template('index_Predict.html')


## Flask Route Setup Part 4

In this part we are going to make the training UI display function for training.

This functino renders the training page. This function will check if the model is being changed or is a file uplaoded for training. It will redirect to the loading screen for training once everything is validated.
The reason why we do not perform the training process in this page is because we want to show an interface while the training process is happening. Otherwise the frontend HTML page will look like it's stuck.

In [ ]:
@app.route('/train',methods=['GET','POST'])
def index_train():
    if session['changedmodel'] == 'True':
        session['changedmodel'] = 'False'
        return render_template('index_Train.html',modelChanged='True')
    elif request.method == 'POST':
        file_to_train = request.files['file']
        if file_to_train and allowed_file(file_to_train.filename):
            filename = secure_filename(file_to_train.filename)
            app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER_TRAIN
            file_to_train.save(os.path.join(app.config['UPLOAD_FOLDER'],filename))
            session['trainingFile'] = file_to_train.filename
            return redirect(url_for('load_Train'))
        elif request.form['change'] == 'Change Model':
            return redirect(url_for('change_model'))
    return render_template('index_Train.html')



## Flask Route Setup Part 5

Finally we get to the training part. This function we are going to make will take care of the training process, getting the classification report, and redirecting to the results page.

In [ ]:
@app.route('/training_in_progress', methods=['GET', 'POST'])
def load_Train():
    global newReport, currentReport,temp_model
    temp_model = NN_model()
    if request.method == 'POST':
        latest_report = ''
        try:
            latest_report = temp_model.train(session['trainingFile'])
        except:
            errorString = 'Unable to generate a report based on the training data. Training data is invalid. Please upload another set of training data.'
            log.error(errorString)
            session['errorInfo'] = errorString
            session['errorOc'] = True
            return redirect(url_for('index_train'))

        if model is not None:
            previous_report = model.get_report()
        else:
            previous_report = None
            
            
        if previous_report == None:
            currentReport = [{"No Model has been trained before": "This is the first training of the model."}]
        else:
            currentReport = previous_report
            
        newReport = latest_report
        session['fileChosen'] = session['trainingFile']
        app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
        log.info('Training complete.')
        return redirect(url_for('train_res'))
    else:
        return render_template('loading.html')

## Flask Route Setup Part 6

In this part we are going to make a function that displays the training result. If the user accepts the results then we will redirect to the loading page where the changes to the active model will be applied. If the user declines the training results, then the temporary model object will be discarded.

In [ ]:
@app.route('/train_result', methods=['GET','POST'])
def train_res():
    global temp_model
    log.info('Training results rendered.')
    try:
        if request.method == 'POST':
            #These statements check to see the decision of the user
            if request.form['accept'] == 'Accept Changes':
                log.info('Changes accepted.')
                return redirect(url_for('load_accept'))
            elif request.form['accept'] == 'Discard Changes':
                #If the user discards the changes, this line will reset the temporary model
                temp_model = None
                log.info('Changes rejected. Discarding new model.')
                return redirect(url_for('index_train'))
            else:
                return render_template('Train_result.html', newjsonTable=newReport, currentjsonTable=currentReport)
    except:
        errorString = 'Unable to gather model training results. Returning to training page.'
        log.error(errorString)
        session['errorOc'] = True
        session['errorInfo'] = errorString
        return render_template('Train_result.html', newjsonTable=newReport, currentjsonTable=currentReport, errorOcurred='True', errorInfo=errorString)
    return render_template('Train_result.html', newjsonTable=newReport, currentjsonTable=currentReport)

## Flask Route Setup Part 7

In this part we will implement the function that will apply the changes once you accept the training results. It will update the picle file to the newest model. Afterwards, this function will redirect you back to the training page.

In [ ]:
@app.route('/loading', methods=['GET', 'POST'])
def load_accept(filename_load="nothing"):
	global temp_model,model
	if request.method != 'GET':
		filename_load = session['trainingFile']
	if request.method == 'GET':
		return render_template('accepted.html')
	elif request.method == 'POST' and filename_load != 'nothing':
		log.info('Executing saving sequence.')
		model = temp_model
		temp_model = None
		file_pickle = open(fname,'wb')
		pickled = pkl.dump(model,file_pickle)
		file_pickle.close()
		return redirect(url_for('index_train'))
	return render_template('accepted.html')

# Congrats! You have completed all the coding part for the Flask App!!

## Now you can use the previous file "run.py" that you made to run the Flask Application by calling the command below.

In [ ]:
python run.py

# Now you are ready to access your app from your browser!

Click this!
[127.0.0.1:5000](http://127.0.0.1:5000)